In [ ]:
import os
import sys

dir = os.path.abspath('')
while not dir.endswith('ardt'): dir = os.path.dirname(dir)
if not dir in sys.path: sys.path.append(dir)

In [ ]:
import io
import time

import paramiko

from utils.helpers import find_root_dir

from private_keys import KNUCKLES_PASSWORD, BLAZE_PRIVATE_KEY, GITHUB_PRIVATE_KEY

In [ ]:
SCRIPT_TO_RUN = 'run_experiment_test.sh'  # 'run_experiment_1.sh', 'run_experiment_2.sh', etc.

## Connecting to Blaze

In [ ]:
# Private key
k = BLAZE_PRIVATE_KEY.strip()
padding = len(k) % 4
if padding > 0:
    k += "=" * (4 - padding)
private_key_file = io.StringIO(k)
pkey = paramiko.RSAKey.from_private_key(private_key_file)

# Establish SSH connection to the first server (knuckles)
knuckles = paramiko.SSHClient()
knuckles.set_missing_host_key_policy(paramiko.AutoAddPolicy())
knuckles.connect('knuckles.cs.ucl.ac.uk', username='amarques', password=KNUCKLES_PASSWORD)

# Use this connection to establish a second SSH connection to the second server (blaze)
# Paramiko Transport is used here for nested ssh
transport = knuckles.get_transport()
dest_addr = ('blaze.cs.ucl.ac.uk', 22) # IP and port
local_addr = ('knuckles.cs.ucl.ac.uk', 22) # IP and port
jump = transport.open_channel("direct-tcpip", dest_addr, local_addr)

blaze = paramiko.SSHClient()
blaze.set_missing_host_key_policy(paramiko.AutoAddPolicy())
blaze.connect('blaze.cs.ucl.ac.uk', username='amarques', sock=jump, pkey=pkey)

## Set up environment, code, files

In [ ]:
HOME_DIR = "/cs/student/msc/dsml/2022/amarques"
ARDT_DIR = find_root_dir()

In [ ]:
blaze.exec_command('source /usr/local/cuda/CUDA_VISIBILITY.csh')
time.sleep(20)

blaze.exec_command('source /opt/Python/Python-3.10.1_Setup.csh')
time.sleep(20)


In [ ]:
blaze.exec_command('mkdir ardt-env')
time.sleep(5)

blaze.exec_command(f'python3.10 -m venv {HOME_DIR}/ardt-env/ardt')
time.sleep(20)

blaze.exec_command(f'source {HOME_DIR}/ardt-env/ardt/bin/activate.csh')
time.sleep(5)

In [ ]:
blaze.exec_command(f'rm -rf {HOME_DIR}/action-robust-decision-transformer')
blaze.exec_command(f'git clone https://{GITHUB_PRIVATE_KEY}@github.com/afonsosamarques/action-robust-decision-transformer.git')
time.sleep(120)

In [ ]:
blaze.exec_command('pip3 install --upgrade pip')
time.sleep(10)

blaze.exec_command(f'pip3 install -r {HOME_DIR}/action-robust-decision-transformer/requirements.txt')
time.sleep(180)

In [ ]:
blaze.exec_command(f'mkdir {HOME_DIR}/action-robust-decision-transformer/ardt/eval-outputs')
time.sleep(3)

blaze.exec_command(f'mkdir {HOME_DIR}/action-robust-decision-transformer/ardt/eval-outputs-pipeline')
time.sleep(3)

blaze.exec_command(f'mkdir {HOME_DIR}/action-robust-decision-transformer/ardt/eval-outputs-test')
time.sleep(3)

blaze.exec_command(f'mkdir {HOME_DIR}/action-robust-decision-transformer/ardt/agents')
time.sleep(3)

blaze.exec_command(f'mkdir {HOME_DIR}/action-robust-decision-transformer/ardt/agents-pipeline')
time.sleep(3)

blaze.exec_command(f'mkdir {HOME_DIR}/action-robust-decision-transformer/ardt/agents-test')
time.sleep(3)

blaze.exec_command(f'mkdir {HOME_DIR}/action-robust-decision-transformer/ardt/wandb')
time.sleep(3)

blaze.exec_command(f'mkdir {HOME_DIR}/action-robust-decision-transformer/ardt/wandb-json')
time.sleep(3)

In [ ]:
src = f'{ARDT_DIR}/access_tokens.py'
os.system(f"rsync -av {src} amarques@knuckles.cs.ucl.ac.uk:{HOME_DIR}")
time.sleep(5)
blaze.exec_command(f'mv {HOME_DIR}/access_tokens.py {HOME_DIR}/action-robust-decision-transformer/ardt')
time.sleep(5)

src = f'{ARDT_DIR}/all-notebooks/clusters-nbs/other-run-scripts/{SCRIPT_TO_RUN}'
os.system(f"rsync -av {src} amarques@knuckles.cs.ucl.ac.uk:{HOME_DIR}")
time.sleep(5)
blaze.exec_command(f'chmod +x {HOME_DIR}/{SCRIPT_TO_RUN}')

In [ ]:
src = f'{ARDT_DIR}/ardt/datasets'
os.system(f"rsync -av {src} amarques@knuckles.cs.ucl.ac.uk:{HOME_DIR}")

## Run script and close connections

In [ ]:
blaze.exec_command(f'cp -r {HOME_DIR}/datasets {HOME_DIR}/action-robust-decision-transformer/ardt && {HOME_DIR}/{SCRIPT_TO_RUN}')

In [ ]:
time.sleep(600)
blaze.close()
knuckles.close()